https://emap.pcsc.com.tw/emap.aspx

-> 
https://emap.pcsc.com.tw/EMapSDK.aspx

In [2]:
import requests
import pandas as pd
from lxml import etree
from io import BytesIO

In [2]:
import requests
url = 'https://emap.pcsc.com.tw/EMapSDK.aspx'
formdata = {'commandid': 'GetTown',
            'cityid': '01',
           }

res = requests.post(url = url,data=formdata)
res

<Response [200]>

In [3]:
res.text
# 確認是xml格式

'<?xml version="1.0" encoding="utf-8"?><iMapSDKOutput><MessageID>00000</MessageID><CommandID>GetTown</CommandID><Status>連線成功</Status><TimeStamp>2020/9/2 下午 09:59:01</TimeStamp><GeoPosition><TownID>01</TownID><TownName>松山區</TownName><X>121577218</X><Y>25049837</Y></GeoPosition><GeoPosition><TownID>02</TownID><TownName>信義區</TownName><X>121567161</X><Y>25033147</Y></GeoPosition><GeoPosition><TownID>03</TownID><TownName>大安區</TownName><X>121534593</X><Y>25026482</Y></GeoPosition><GeoPosition><TownID>04</TownID><TownName>中山區</TownName><X>121533655</X><Y>25064427</Y></GeoPosition><GeoPosition><TownID>05</TownID><TownName>中正區</TownName><X>121518245</X><Y>25032251</Y></GeoPosition><GeoPosition><TownID>06</TownID><TownName>大同區</TownName><X>121515830</X><Y>25066142</Y></GeoPosition><GeoPosition><TownID>07</TownID><TownName>萬華區</TownName><X>121499745</X><Y>25034807</Y></GeoPosition><GeoPosition><TownID>08</TownID><TownName>文山區</TownName><X>121570280</X><Y>24989800</Y></GeoPosition><GeoPosition><To

In [33]:
# xml的正確顯示方式
res.content

b'<?xml version="1.0" encoding="utf-8"?><iMapSDKOutput><MessageID>00000</MessageID><CommandID>GetTown</CommandID><Status>\xe9\x80\xa3\xe7\xb7\x9a\xe6\x88\x90\xe5\x8a\x9f</Status><TimeStamp>2020/9/2 \xe4\xb8\x8b\xe5\x8d\x88 09:59:01</TimeStamp><GeoPosition><TownID>01</TownID><TownName>\xe6\x9d\xbe\xe5\xb1\xb1\xe5\x8d\x80</TownName><X>121577218</X><Y>25049837</Y></GeoPosition><GeoPosition><TownID>02</TownID><TownName>\xe4\xbf\xa1\xe7\xbe\xa9\xe5\x8d\x80</TownName><X>121567161</X><Y>25033147</Y></GeoPosition><GeoPosition><TownID>03</TownID><TownName>\xe5\xa4\xa7\xe5\xae\x89\xe5\x8d\x80</TownName><X>121534593</X><Y>25026482</Y></GeoPosition><GeoPosition><TownID>04</TownID><TownName>\xe4\xb8\xad\xe5\xb1\xb1\xe5\x8d\x80</TownName><X>121533655</X><Y>25064427</Y></GeoPosition><GeoPosition><TownID>05</TownID><TownName>\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80</TownName><X>121518245</X><Y>25032251</Y></GeoPosition><GeoPosition><TownID>06</TownID><TownName>\xe5\xa4\xa7\xe5\x90\x8c\xe5\x8d\x80</TownNam

In [32]:
type(res.content)

bytes

In [34]:
# 以io和lxml解析xml
from lxml import etree
from io import BytesIO

# res.content -> (BytesIO) -> file -> (etree.parse) -> tree
file = BytesIO(res.content)
tree = etree.parse(file)
tree

In [43]:
# 用xpath的方式找資料
# 路徑'/iMapSDKOutput/MessageID' or '//MessageID'

tree.xpath('/iMapSDKOutput/MessageID')[0].text


'00000'

In [44]:
tree.xpath('//TownName')[0].text

'松山區'

In [48]:
# 台北市所有行政區
for i,ele in enumerate(tree.xpath('//TownName')):
    print(i+1,ele.text)

1 松山區
2 信義區
3 大安區
4 中山區
5 中正區
6 大同區
7 萬華區
8 文山區
9 南港區
10 內湖區
11 士林區
12 北投區


# 抓行政區所有的店面資訊

In [3]:
import requests
import pandas as pd
from lxml import etree
from io import BytesIO

url = 'https://emap.pcsc.com.tw/EMapSDK.aspx'
county = '台北市'
dist = '北投區'
formdata = {'commandid': 'SearchStore',
            'city': county,
            'town': dist,
           }

res = requests.post(url = url,data=formdata)
file = BytesIO(res.content)
tree = etree.parse(file)

In [54]:
POIIDs = [ele.text for ele in tree.xpath('//POIID')]
Xs = [ele.text for ele in tree.xpath('//X')]
Ys = [ele.text for ele in tree.xpath('//Y')]
Telnos = [ele.text for ele in tree.xpath('//Telno')]
Address = [ele.text for ele in tree.xpath('//Address')]

In [56]:
import pandas as pd

df = pd.DataFrame({"縣市":county,
                   "行政區":dist,
                   "店ID":POIIDs,
                   "X":Xs,
                   "Y":Ys,
                   "電話":Telnos,
                   "地址":Address
                  })
df

,縣市,行政區,店ID,X,Y,電話,地址
0,台北市,北投區,851721,121468456,25128150,(02)28984000,台北市北投區中央北路四段513號
1,台北市,北投區,187552,121501056,25130534,(02)28963137,台北市北投區中央南路一段114號
2,台北市,北投區,128874,121484900,25137918,(02)28953832,台北市北投區中央北路三段55號57號1樓
3,台北市,北投區,155496,121508172.363924,25125606.2108754,(02)28946735,台北市北投區公館路259號1樓
4,台北市,北投區,936244,121503115,25133059,(02)28965936,台北市北投區公館路31號
5,台北市,北投區,980436,121523837,25113726,(02)28226934,台北市北投區富貴一路3號1樓
6,台北市,北投區,135975,121506171.862779,25135692.7461313,(02)28941650,台北市北投區溫泉路73巷5號地下二樓之1
7,台北市,北投區,136141,121501790,25139090,(02)28941151,台北市北投區中和街222號
8,台北市,北投區,962483,121515502,25115351,(02)28208526,台北市北投區石牌路二段8號
9,台北市,北投區,165839,121502749,25118635,(02)28203682,台北市北投區承德路七段378號


In [4]:
# 全台縣市
counties = {'01':"台北市",
            '03':"新北市",
            '04':"桃園市",
            '08':"台中市",
            '15':"台南市",
            '17':"高雄市",
            '06':"新竹縣",
            '07':"苗栗縣",
            '10':"彰化縣",
            '11':"南投縣",
            '12':"雲林縣",
            '14':"嘉義縣",
            '19':"屏東縣",
            '20':"宜蘭縣",
            '21':"花蓮縣",
            '22':"台東縣",
            '23':"澎湖縣",
            '25':"金門縣",
            '24':"連江縣",
            '02':"基隆市",
            '05':"新竹市",
            '13':"嘉義市"}
len(counties)

22

In [7]:
# 先取得城市的行政區//TownName

import requests

url = 'https://emap.pcsc.com.tw/EMapSDK.aspx'

df= pd.DataFrame(columns = ["縣市","行政區","店ID","X","Y","電話","地址"])

# 全台縣市
counties = {'01':"台北市",
            '03':"新北市",
            '04':"桃園市",
            '08':"台中市",
            '15':"台南市",
            '17':"高雄市",
            '06':"新竹縣",
            '07':"苗栗縣",
            '10':"彰化縣",
            '11':"南投縣",
            '12':"雲林縣",
            '14':"嘉義縣",
            '19':"屏東縣",
            '20':"宜蘭縣",
            '21':"花蓮縣",
            '22':"台東縣",
            '23':"澎湖縣",
            '25':"金門縣",
            '24':"連江縣",
            '02':"基隆市",
            '05':"新竹市",
            '13':"嘉義市"}


for i in range(1,len(counties)):
    if i < 10:
        cityid = "0"+str(i)
    else:
        cityid = i 

    try:
        print(cityid,counties[str(cityid)])
#         print(counties[str(cityid)])
        county = counties[str(cityid)]
        get_dist_formdata = {'commandid': 'GetTown',
            'cityid': cityid,
           }
        res = requests.post(url = url,data=get_dist_formdata)
        file = BytesIO(res.content)
        tree = etree.parse(file)

        #所有行政區
        dists = [ele.text for ele in tree.xpath('//TownName')]
#         print(dists)
        for dist in dists:
#             print(county,dist)
            get_store_info_formdata = {'commandid': 'SearchStore',
                'city': county,
                'town': dist,
               }
            res = requests.post(url = url,data=get_store_info_formdata)
            file = BytesIO(res.content)
            tree = etree.parse(file)
        
            POIIDs = [ele.text for ele in tree.xpath('//POIID')]
            POINames = [ele.text for ele in tree.xpath('//POIName')]
            Xs = [ele.text for ele in tree.xpath('//X')]
            Ys = [ele.text for ele in tree.xpath('//Y')]
            Telnos = [ele.text for ele in tree.xpath('//Telno')]
            Address = [ele.text for ele in tree.xpath('//Address')]
            df_temp = pd.DataFrame({"縣市":county,
                                    "行政區":dist,
                                    "店ID":POIIDs,
                                    "店名":POINames,
                                    "X":Xs,
                                    "Y":Ys,
                                    "電話":Telnos,
                                    "地址":Address
                              })
            df = pd.concat([df,df_temp],ignore_index=True)
            print(f'{counties[str(cityid)]}{dist}完成')
    except:
        pass
df

01 台北市


/Users/Martin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



台北市松山區完成
台北市信義區完成
台北市大安區完成
台北市中山區完成
台北市中正區完成
台北市大同區完成
台北市萬華區完成
台北市文山區完成
台北市南港區完成
台北市內湖區完成
台北市士林區完成
台北市北投區完成
02 基隆市
基隆市中正區完成
基隆市七堵區完成
基隆市暖暖區完成
基隆市安樂區完成
基隆市中山區完成
基隆市仁愛區完成
基隆市信義區完成
03 新北市
新北市板橋區完成
新北市三重區完成
新北市中和區完成
新北市永和區完成
新北市新莊區完成
新北市新店區完成
新北市樹林區完成
新北市土城區完成
新北市蘆洲區完成
新北市汐止區完成
新北市鶯歌區完成
新北市三峽區完成
新北市淡水區完成
新北市瑞芳區完成
新北市五股區完成
新北市泰山區完成
新北市林口區完成
新北市深坑區完成
新北市石碇區完成
新北市坪林區完成
新北市三芝區完成
新北市石門區完成
新北市八里區完成
新北市平溪區完成
新北市雙溪區完成
新北市貢寮區完成
新北市金山區完成
新北市萬里區完成
新北市烏來區完成
04 桃園市
桃園市桃園區完成
桃園市中壢區完成
桃園市八德區完成
桃園市平鎮區完成
桃園市龜山區完成
桃園市大溪區完成
桃園市楊梅區完成
桃園市蘆竹區完成
桃園市大園區完成
桃園市龍潭區完成
桃園市新屋區完成
桃園市觀音區完成
桃園市復興區完成
05 新竹市
新竹市東區完成
新竹市北區完成
新竹市香山區完成
06 新竹縣
新竹縣竹北市完成
新竹縣竹東鎮完成
新竹縣新埔鎮完成
新竹縣關西鎮完成
新竹縣湖口鄉完成
新竹縣新豐鄉完成
新竹縣芎林鄉完成
新竹縣橫山鄉完成
新竹縣北埔鄉完成
新竹縣寶山鄉完成
新竹縣峨眉鄉完成
07 苗栗縣
苗栗縣苗栗市完成
苗栗縣竹南鎮完成
苗栗縣後龍鎮完成
苗栗縣頭份市完成
苗栗縣造橋鄉完成
苗栗縣三灣鄉完成
苗栗縣頭屋鄉完成
苗栗縣南庄鄉完成
苗栗縣公館鄉完成
苗栗縣西湖鄉完成
苗栗縣通霄鎮完成
苗栗縣苑裡鎮完成
苗栗縣銅鑼鄉完成
苗栗縣大湖鄉完成
苗栗縣三義鄉完成
苗栗縣卓蘭鎮完成
苗栗縣泰安鄉完成
08 台中市
台中市豐原區完成
台中市中區完成
台中市大里區完成
台中市東區完成
台中市南區完成
台中市太平區完成
台中市和平區完成
台中市西區完成
台中市北區完成
台中市東勢區完成
台中市后里區完成
台中市西屯區完成
台中市外埔區完成
台中市南屯區完成
台中市北

,X,Y,地址,店ID,店名,縣市,行政區,電話
0,121548287.390895,25056390.9685318,台北市松山區敦化北路168號B2,170945,上弘,台北市,松山區,(02)25472928
1,121549433,25050944,台北市松山區南京東路四段2號1樓,200376,小巨蛋,台北市,松山區,(02)25774625
2,121552737,25048396,台北市松山區八德路三段27號,960661,中崙,台北市,松山區,(02)25774806
3,121552850,25050888,台北市松山區北寧路66號,158923,北體,台北市,松山區,(02)25771113
4,121551158,25048086,台北市松山區八德路三段20-2號,201302,台場,台北市,松山區,(02)25794824
...,...,...,...,...,...,...,...,...
5848,121612685,24090965,花蓮縣秀林鄉景美村加灣178之20號,194239,加灣,花蓮縣,秀林鄉,(03)8264673
5849,121747104,24309346,花蓮縣秀林鄉和平村和平街156號,978260,泰雅,花蓮縣,秀林鄉,(03)8681479
5850,121650659,24156835,花蓮縣秀林鄉崇德95-6號,114288,崇德盈,花蓮縣,秀林鄉,(03)8621776
5851,121653235,24166806,花蓮縣秀林鄉崇德村海濱3號3之1號1樓,207540,達基力,花蓮縣,秀林鄉,(03)8621437


In [8]:
df.to_csv('7_11全台店面資訊.csv')